In [9]:
Pkg.add.(["JuMP","HiGHS","LinearAlgebra","DelimitedFiles","Random","StatsBase","Graphs","Plots"])

    Updating registry at `C:\Users\MSI\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\MSI\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\MSI\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\MSI\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\MSI\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\MSI\.julia\environments\v1.11\Project.toml`
  [37e2e46d] + LinearAlgebra v1.11.0
  No Changes to `C:\Users\MSI\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
   Installed DelimitedFiles ─ v1.9.1
    Updating `C:\Users\MSI\.julia\environments\v1.11\Project.toml`
  [8bb1440f] + DelimitedFiles v1.9.1
    Updating `C:\Users\MSI\.julia\environments\v1.11\Manifest.toml`
  [8bb1440f] + DelimitedFiles v1.9.1
Precompiling project...
   1246.4 ms  ✓ DelimitedFiles
  1 dependency successfully precompiled

8-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [24]:
using Pkg
using JuMP 
using HiGHS
using LinearAlgebra
using Random
using Graphs
using Plots
using Printf

Random.seed!(42)

const DATA_INPUT_DIR = joinpath("..","data","input")
const OUTPUT_DIR     = joinpath("output","archipelago120")
const INSTANCE_NAME  = "archipelago120"

mkpath(OUTPUT_DIR)

"output\\archipelago120"

In [11]:
function leer_archivo_tsp(path::AbstractString)
    ids = Int[]
    X   = Float64[]
    Y   = Float64[]
    in_coords = false
    open(path, "r") do f
        for ln in eachline(f)
            if occursin("NODE_COORD_SECTION", ln); in_coords = true; continue; end
            if occursin("EOF", ln); break; end
            if in_coords
                tok = split(strip(ln))
                if length(tok) >= 3
                    push!(ids, parse(Int, tok[1]))
                    push!(X, parse(Float64, tok[2]))
                    push!(Y, parse(Float64, tok[3]))
                end
            end
        end
    end
    return ids, hcat(X,Y)
end

function euclid2(a::AbstractVector, b::AbstractVector)
    dx = a[1]-b[1]; dy = a[2]-b[2]
    return sqrt(dx*dx + dy*dy)
end

function matriz_distancias(coords::AbstractMatrix)
    n = size(coords,1)
    D = zeros(Float64, n, n)
    for i in 1:n, j in i+1:n
        d = euclid2(coords[i,:], coords[j,:])
        D[i,j] = d; D[j,i] = d
    end
    return D
end

tsp_path = joinpath(DATA_INPUT_DIR, INSTANCE_NAME*".tsp")
ids, coords = leer_archivo_tsp(tsp_path)
n = length(ids)
distancias = matriz_distancias(coords)
println("Instancia: $INSTANCE_NAME  | n = $n")


Instancia: archipelago120  | n = 120


In [13]:
# helpers DFJ
# Grafo inducido por solución actual
function componentes_from_x(xvals::Matrix{Float64}; thr=0.5)
    g = SimpleGraph(n)
    for i in 1:n, j in i+1:n
        if xvals[i,j] > thr || xvals[j,i] > thr
            add_edge!(g, i, j)
        end
    end
    return connected_components(g)
end

# Agrega corte DFJ para un subconjunto S al menos 1 arco sale de S y 1 entra a S
function agregar_corte!(model::Model, x, S::Vector{Int})
    V = collect(1:n)
    T = setdiff(V, S)
    if !isempty(T) && !isempty(S)
        @constraint(model, sum(x[i,j] for i in S, j in T) >= 1)
        @constraint(model, sum(x[i,j] for i in T, j in S) >= 1)
        return 2
    end
    return 0
end

# Reconstruye tour dirigido desde x (arranca en el nodo 1)
function tour_from_x(xvals::Matrix{Float64})
    nxt = zeros(Int, n)
    for i in 1:n
        j = argmax(xvals[i, :])
        nxt[i] = j
    end
    tour = Int[]
    seen = falses(n)
    cur = 1
    for _ in 1:n
        push!(tour, cur)
        seen[cur] = true
        cur = nxt[cur]
        if seen[cur]; break; end
    end
    return tour
end

function costo_tour(tour::Vector{Int}, D::Matrix{Float64})
    c = 0.0
    for k in 1:length(tour)-1
        c += D[tour[k], tour[k+1]]
    end
    c += D[tour[end], tour[1]]
    return c
end


costo_tour (generic function with 1 method)

In [14]:
### mejora local 2-opt rápida sobre tour
function two_opt!(tour::Vector{Int}, D::Matrix{Float64})
    n = length(tour)
    improved = true
    while improved
        improved = false
        for i in 1:n-3
            for j in i+2:n-1
                a,b = tour[i], tour[i+1]
                c,d = tour[j], tour[j+1]
                Δ = (D[a,c] + D[b,d]) - (D[a,b] + D[c,d])
                if Δ < -1e-9
                    reverse!(tour, i+1, j)
                    improved = true
                end
            end
        end
    end
    return tour
end


two_opt! (generic function with 1 method)

In [15]:
# modelo JuMP 
model = Model(HiGHS.Optimizer)

# Atributos del solver
set_optimizer_attribute(model, "time_limit", 3600.0)
set_optimizer_attribute(model, "threads", 4)
set_optimizer_attribute(model, "parallel", "on")
set_optimizer_attribute(model, "presolve", "on")
set_optimizer_attribute(model, "mip_rel_gap", 0.5)
set_optimizer_attribute(model, "mip_abs_gap", 50.0)
set_optimizer_attribute(model, "mip_max_nodes", 80000)

@variable(model, x[1:n,1:n], Bin)
for i in 1:n
    @constraint(model, x[i,i] == 0)
end

@constraint(model, salida[i in 1:n], sum(x[i,j] for j in 1:n) == 1)
@constraint(model, entrada[i in 1:n], sum(x[j,i] for j in 1:n) == 1)

@objective(model, Min, sum(distancias[i,j] * x[i,j] for i in 1:n, j in 1:n))


28.43132527688427 x[1,2] + 45.66751288388713 x[1,3] + 33.03972398492467 x[1,4] + 2.966042649727066 x[1,5] + 69.64106858025657 x[1,6] + 36.03355204528134 x[1,7] + 40.9241523308669 x[1,8] + 22.758938573668097 x[1,9] + 38.990336546380306 x[1,10] + 46.61866720531595 x[1,11] + 76.31359315351358 x[1,12] + 24.490442237738375 x[1,13] + 65.31925079943888 x[1,14] + 40.78385819169149 x[1,15] + 12.348281540360123 x[1,16] + 50.115343448888034 x[1,17] + 54.09828645160578 x[1,18] + 45.549715487585644 x[1,19] + 12.188990811383887 x[1,20] + 353.06600340446255 x[1,21] + 341.7059987255126 x[1,22] + 343.24594753762204 x[1,23] + 367.60767332850924 x[1,24] + 343.05757513426227 x[1,25] + 392.8445591744907 x[1,26] + 341.55620924527193 x[1,27] + 355.011399155013 x[1,28] + 366.4324110187307 x[1,29] + 368.2163185751006 x[1,30] + 371.83019647145386 x[1,31] + [[...14220 terms omitted...]] + 605.891483225338 x[120,90] + 561.5889239523871 x[120,91] + 565.0023266191033 x[120,92] + 578.2895444965263 x[120,93] + 563.31

In [16]:
# DFJ iterativo agregando cortes hasta 1 componente
iteracion = 1
cortes_totales = 0
mejor_costo = Inf
mejor_x = nothing
t0 = time()

# Pequeños cortes iniciales para evitar 2-ciclos triviales
for i in 1:min(n, 8)
    for j in i+1:min(n, i+2)
        @constraint(model, x[i,j] + x[j,i] <= 1)
    end
end

println("Iniciando DFJ…")
while true
    println("\n--- Iteracion $iteracion ---")
    optimize!(model)

    term = termination_status(model)
    if !(term in (MOI.OPTIMAL, MOI.TIME_LIMIT)) && !has_values(model)
        error("El solver no retorno solucion valida. status = $term")
    end

    xvals = value.(x)
    comps = componentes_from_x(xvals; thr=0.5)
    println(" Componentes encontradas: ", length(comps))

    if length(comps) == 1
        costo = objective_value(model)
        println(" Tour factible DFJ. Costo (solver): ", round(costo, digits=3))
        mejor_costo = costo
        mejor_x = xvals
        break
    end

    # agregar como max 2 cortes por iteración
    nuevos = 0
    for S in comps
        nuevos += agregar_corte!(model, x, S)
        if nuevos >= 2; break; end
    end
    cortes_totales += nuevos
    println(" Cortes agregados en la iteración: $nuevos  |  Total: $cortes_totales")

    # afinar el gap después de algunas iteraciones
    if iteracion == 3
        set_optimizer_attribute(model, "mip_rel_gap", 0.2)
    end

    iteracion += 1
    if time() - t0 > 1200  # 20 minutos tope razonable
        println(" Límite de tiempo global alcanzado, saliendo…")
        break
    end
end

println("\nDFJ terminado en $(round(time()-t0, digits=1))s. Cortes totales: $cortes_totales")


Iniciando DFJ…

--- Iteracion 1 ---
Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 376 rows; 14400 cols; 28952 nonzeros; 14400 integer variables (14400 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 9e+02]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
256 rows, 14280 cols, 28592 nonzeros  0s
256 rows, 14280 cols, 28592 nonzeros  0s

Solving MIP model with:
   256 rows
   14280 cols (14280 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   28592 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds    

In [21]:
# Heurística de Vecino Más Cercano (por si no hay incumbente)
function nearest_neighbor_tour(D::Matrix{Float64}; start::Int=1)
    n = size(D,1)
    unvis = trues(n)
    tour = Int[]
    cur = start
    for _ in 1:n
        push!(tour, cur); unvis[cur] = false
        if any(unvis)
            best = 0
            bestd = Inf
            @inbounds for j in 1:n
                if unvis[j] && D[cur,j] < bestd
                    best = j; bestd = D[cur,j]
                end
            end
            cur = best
        end
    end
    return tour
end

# Si tour_from_x devuelve varios ciclos (longitud < n), completa el tour greedy
function completar_tour_si_falta(tour::Vector{Int}, D::Matrix{Float64})
    n = size(D,1)
    if length(tour) == n
        return tour
    end
    pend = setdiff(1:n, tour)
    cur = tour[end]
    while !isempty(pend)
        # escoger el más cercano al 'cur'
        idx = 1
        best = pend[idx]
        bestd = D[cur, best]
        for k in 2:length(pend)
            if D[cur, pend[k]] < bestd
                idx = k; best = pend[k]; bestd = D[cur, pend[k]]
            end
        end
        push!(tour, best)
        cur = best
        deleteat!(pend, idx)
    end
    return tour
end


completar_tour_si_falta (generic function with 1 method)

In [22]:
# si no hubo incumbente (muy raro), usa heurística NN
tour_dfj = if mejor_x === nothing
    @warn "DFJ terminó sin incumbente; usando heurística de vecino más cercano."
    nearest_neighbor_tour(distancias; start=1)
else
    t = tour_from_x(mejor_x)
    completar_tour_si_falta(t, distancias)
end

costo_dfj_real = costo_tour(tour_dfj, distancias)

tour_2opt = copy(tour_dfj)
two_opt!(tour_2opt, distancias)
costo_2opt = costo_tour(tour_2opt, distancias)

usar_2opt = costo_2opt + 1e-6 < costo_dfj_real
tour_final = usar_2opt ? tour_2opt : tour_dfj
costo_final = usar_2opt ? costo_2opt : costo_dfj_real

println("\n=======================")
println("Costo DFJ real  : ", round(costo_dfj_real, digits=3))
println("Costo 2-opt     : ", round(costo_2opt, digits=3))
println("DECISIÓN        : ", usar_2opt ? "Usar 2-opt" : "Usar DFJ")
println("Costo FINAL     : ", round(costo_final, digits=3))


┌ Warning: DFJ terminó sin incumbente; usando heurística de vecino más cercano.
└ @ Main c:\Users\MSI\Desktop\info\Escritorio\UVG\8vo Semestre\Modelacion y Sinulacion\Proyectos\Proyecto 1\PROY1_MODSIM\propio\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:3



Costo DFJ real  : 3939.288
Costo 2-opt     : 3574.705
DECISIÓN        : Usar 2-opt
Costo FINAL     : 3574.705


In [25]:
# guardar TOUR, costo y figura
function write_tsplib_tour(path::AbstractString, ids_1based::Vector{Int}; name="DFJ_SOLUTION")
    open(path, "w") do f
        println(f, "NAME: $name")
        println(f, "TYPE: TOUR")
        println(f, "DIMENSION: $(length(ids_1based))")
        println(f, "TOUR_SECTION")
        for v in ids_1based; println(f, v); end
        println(f, -1)
        println(f, "EOF")
    end
end

# ids reales (TSPLIB) en orden del tour
ids_tour_1based = [ids[i] for i in tour_final]

tour_path  = joinpath(OUTPUT_DIR, "tour_dfj_$(INSTANCE_NAME).tour")
costo_path = joinpath(OUTPUT_DIR, "costo_dfj_$(INSTANCE_NAME).txt")
write_tsplib_tour(tour_path, ids_tour_1based; name="$(INSTANCE_NAME)_DFJ")
open(costo_path, "w") do f; println(f, @sprintf("%.6f", costo_final)); end

# figura
xs, ys = coords[:,1], coords[:,2]
ord = vcat(tour_final, tour_final[1])
p = plot(xs[ord], ys[ord], lw=2, marker=:circle, legend=false, title="TSP $(INSTANCE_NAME) – DFJ")
scatter!(xs, ys)
png_path = joinpath(OUTPUT_DIR, "tour_dfj_$(INSTANCE_NAME).png")
savefig(p, png_path)

println("\nArchivos guardados:")
println(" - ", tour_path)
println(" - ", costo_path)
println(" - ", png_path)



Archivos guardados:
 - output\archipelago120\tour_dfj_archipelago120.tour
 - output\archipelago120\costo_dfj_archipelago120.txt
 - output\archipelago120\tour_dfj_archipelago120.png
